In [4]:
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np 
from utils.imgs_in_extent import imgs_in_extent
from glob import glob
from utils.ransac_fitting import ransac_fitting
import rasterio as rio
from rasterio.merge import merge
from rasterio.mask import mask
import warnings
import pandas as pd


In [9]:
path_rgi60_setp_area_le_4 = 'data/land-cover/rgi60/rgi60_setp_area_le_4.shp'
dir_dif_map = 'data/aster-stereo/tiles-dif-map'
paths_dif_map = glob(dir_dif_map + '/tile_??_??.tif')
path_result_glacier = 'data/ele-change-result/ele_change_glaciers.pkl'
path_srtm = 'data/dem-data/srtm-c/SRTMGL1_E.tif'


In [6]:
rgi60_setp_sel = gpd.read_file(path_rgi60_setp_area_le_4)
rgi60_setp_sel.head()


,RGIId,Area,CenLon,CenLat,geometry
0,RGI60-13.00643,28.533,94.928018,30.607432,"POLYGON ((94.97412 30.60416, 94.97372 30.60385..."
1,RGI60-13.00761,36.251,94.636865,30.632809,"POLYGON ((94.57537 30.65285, 94.57536 30.65286..."
2,RGI60-13.00763,5.385,94.565846,30.631692,"POLYGON ((94.57356 30.64344, 94.57359 30.64346..."
3,RGI60-13.00777,5.293,94.384270,30.624226,"POLYGON ((94.39529 30.63960, 94.39546 30.63984..."
4,RGI60-13.00788,34.120,94.383484,30.581293,"POLYGON ((94.43347 30.58483, 94.43378 30.58471..."


In [7]:
rgi_id = np.array(rgi60_setp_sel.RGIId)
area_glaciers = np.array(rgi60_setp_sel.Area)
x_years = np.arange(23)
years = np.array([str(year) for year in range(2000, 2023)])


In [18]:
rate_glaciers = []
dif_map_glaciers = []
ele_change_glaciers = []
ele_change_fitting_glaciers = []
srtm_glaciers = []
## obtain the dif_maps of the glaicer reigon.
for i_gla in range(rgi60_setp_sel.shape[0]):
    # i_gla = 50
    print('i_gla:', i_gla)
    geo_gla = rgi60_setp_sel.iloc[i_gla]['geometry'] 
    bounds = geo_gla.bounds
    extent = [bounds[0], bounds[2], bounds[1], bounds[3]]
    dif_map_sel = imgs_in_extent(paths_img=paths_dif_map, extent=extent)
    print('number of selected dif_map:', len(dif_map_sel))
    # dif_map
    dif_map = rio.open(dif_map_sel[0])
    if len(dif_map_sel)>1:
        dif_map_mosaic, dif_map_mosaic_trans = merge(dif_map_sel)
        meta_mosaic = dif_map.meta.copy()
        meta_mosaic.update({'driver': 'GTiff',
                            'height': dif_map_mosaic.shape[1],
                            'width': dif_map_mosaic.shape[2],
                            'transform': dif_map_mosaic_trans,                       
                            })
        with rio.open('dif_map_mosaic.tif', 'w', **meta_mosaic) as m:
            m.write(dif_map_mosaic)
        dif_map = rio.open('dif_map_mosaic.tif')
    dif_map_gla_masked, dif_map_gla_masked_trans = mask(dif_map, [geo_gla], filled=False, crop=True)
    dif_map_gla_masked = dif_map_gla_masked.filled(np.nan)
    ## srtm dem
    srtm_dem = rio.open(path_srtm)
    srtm_dem_gla_masked, srtm_dem_gla_masked_trans = mask(srtm_dem, [geo_gla], filled=False, crop=True)
    srtm_dem_gla_masked = srtm_dem_gla_masked.filled(np.nan)
    srtm_dem_gla_masked = np.reshape(srtm_dem_gla_masked, srtm_dem_gla_masked.shape[1:])

    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        dif_maps_gla_mean = np.nanmean(dif_map_gla_masked, axis=(1,2))
    mean_filtered, mean_fitting, rate = ransac_fitting(x=x_years, y=dif_maps_gla_mean, thre_mask=50)
    print(rate)
    rate_glaciers.append(rate)
    dif_map_glaciers.append(dif_map_gla_masked)
    ele_change_glaciers.append(dif_maps_gla_mean)
    ele_change_fitting_glaciers.append(mean_fitting)
    srtm_glaciers.append(srtm_dem_gla_masked)
if os.path.exists('dif_map_mosaic.tif'): os.remove('dif_map_mosaic.tif')



i_gla: 0
number of selected dif_map: 1
-1.1985477121881365
i_gla: 1
number of selected dif_map: 1
-0.4134264304396809
i_gla: 2
number of selected dif_map: 1
-2.0273820561264193
i_gla: 3
number of selected dif_map: 1
0.9909940510657345
i_gla: 4
number of selected dif_map: 1
-1.0698805607843322
i_gla: 5
number of selected dif_map: 1
-0.8120210897820509
i_gla: 6
number of selected dif_map: 1
-1.1469749806018974
i_gla: 7
number of selected dif_map: 1
-1.363765780131022
i_gla: 8
number of selected dif_map: 1
-0.21386237457411178
i_gla: 9
number of selected dif_map: 1
-1.065234381889904
i_gla: 10
number of selected dif_map: 1
0.07782170805009103
i_gla: 11
number of selected dif_map: 1
-3.203393129619756
i_gla: 12
number of selected dif_map: 1
-0.8026547307465353
i_gla: 13
number of selected dif_map: 1
1.8416242963948817
i_gla: 14
number of selected dif_map: 1
1.0974037343314282
i_gla: 15
number of selected dif_map: 1
-0.7886195658117783
i_gla: 16
number of selected dif_map: 1
-0.385943296249

In [20]:
result_df = pd.DataFrame({'rgi_id': rgi_id,
                   'ele_change': ele_change_glaciers,
                   'ele_change_fitting': ele_change_fitting_glaciers,
                   'rate': rate_glaciers,
                   'gla_area':area_glaciers,
                   'dif_map': dif_map_glaciers,
                   'geometry':rgi60_setp_sel['geometry'],
                   'srtm_dem': srtm_glaciers,
                   })


### Write out

In [21]:
if os.path.exists(path_result_glacier): os.remove(path_result_glacier)
result_df.to_pickle(path_result_glacier)

